<a href="https://colab.research.google.com/github/heromiya/GEE-LCM/blob/local-jupyter/trainingDataPointGen_GEEUpload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf GEE-LCM && git clone https://github.com/heromiya/GEE-LCM.git

In [ ]:
!apt update && apt install spatialite-bin grass parallel zip gdal-bin

In [ ]:
# bash merge-traningdata.sh -b [Starting year] -e [Ending year] --pmin [minimum path number] --pmax [maximum path number] --rmin [minimum row number] --rmax [maximum row number] -1 [# of samples for Class 1 (waterbody)] -2 [# of samples for Class 2 (vegetation)] -3 [# of samples for Class 3 (bare land)] -4 [# of samples for Class 4 (built-up)]
# 0.00025 is equal to 30 meters.
!cd GEE-LCM && bash merge-trainingdata.sh -b 1900 -e 2000 --pmin 0 --pmax 999 --rmin 0 --rmax 999 -r 0.00025 -1 2500 -2 2500 -3 2500 -4 2500

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

# Please have the Google Bucket credential JSON file in your drive

In [ ]:
import os
import time
from pprint import pprint
from google.cloud import storage

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'land-cover-mapping.json'

ts = time.time()
timestamp = int(ts)

UserName = "ZahidWani"
storage_client = storage.Client()
blob_name = UserName + '_' + str(timestamp) + '/'

project_name = 'land-cover-mapping-340607'
bucket_name = 'landcovermapping'

gcs_client = storage.Client(project=project_name)
bucket = gcs_client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_string('', content_type='application/x-www-form-urlencoded;charset=UTF-8')

def upload_to_bucket(blob_name, file_path, bucket_name):
    try:
        bucket = storage_client.get_bucket(bucket_name)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(file_path)
        print("Shapefile {} Uploaded.".format(blob_name))
#         return True
    except Exception as e:
        print(e)
        return False
    
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.delete()

    print("Blob {} deleted.".format(blob_name))

In [ ]:
file_path = r'GEE-LCM'
name1 = 'gt-pt.prj'
name2 = 'gt-pt.shp'
name3 = 'gt-pt.shx'
name4 = 'gt-pt.dbf'

upload_to_bucket(blob.name + name1, os.path.join(file_path, name1), bucket_name)
upload_to_bucket(blob.name + name2, os.path.join(file_path, name2), bucket_name)
upload_to_bucket(blob.name + name3, os.path.join(file_path, name3), bucket_name)
upload_to_bucket(blob.name + name4, os.path.join(file_path, name4), bucket_name)

In [ ]:
!earthengine upload table --asset_id=users/zahidwani/ft-pt-10105025 gs://landcovermapping/ZahidWani_1644997179/gt-pt.shp